# INTRODUCTION

League of Legends (LOL) is a popular online MOBA game developed by Riot Games. 10 Players will form 2 teams and compete with each other and the goal of each team is destroy their enemy's nexus.  
  
Each LOL match will produce tons of datas and what we want to do is to predict the game result (which team will win in the end) basing on those datas. Since LOL players are millions and matches are holded every minute, it would be valuable for players to predict the outcomes of macthes and also really interesting.

In [1]:
from cassiopeia import Summoner
from cassiopeia import Match
from cassiopeia import get_champion_mastery
from cassiopeia import Champions
from cassiopeia import SummonerSpells
from cassiopeia import GameType
from cassiopeia import Role
from cassiopeia import Lane
from cassiopeia import GameMode
import pandas as pd
import numpy as np
import math

We choose cassiopeia as a tool to collect LOL match datas, it provides easy Python access to Riot api so we don't need to assemble http requests ourselves and makes life easier.

Then we have a class called SimpleMatch because it simplify the match data and only record the fields that we think is most useful to our model training.

In [2]:
class SimpleMatch:

    def __init__(self, match, player):
        # if (len(match.blue_team.participants) < 5 or len(match.red_team.participants) < 5) or (match.blue_team.participants[0].lane == None):
        try:
            if match.mode != GameMode.classic:
                self.valid = False
                return
            else:
                self.valid = True

            # definition
            role_index = {Role.top: 0, Role.middle: 1, Role.jungle: 2, Role.adc: 3, Role.support: 4}
            lane_index = {Lane.top_lane: 0, Lane.mid_lane: 1, Lane.jungle: 2, Lane.bot_lane: 3}
            # get common infos
            self.all_champions = Champions(region="NA")
            self.total_champions_number = len(self.all_champions)
            self.player_number_perTeam = 5
            self.team_number = 2
            self.all_spells = SummonerSpells(region="NA", type=GameType.matched)
            self.total_spells_number = len(self.all_spells)
            # save timeline for early game data
            self.timeline = match.timeline.frames
            # type check
            BLUE = 1
            RED = 0
            if (not isinstance(match, Match)):
                raise Exception("Invalid init type of SimpleMatch: ", match)
            if (not isinstance(player, Summoner)):
                raise Exception("Invalid init type of SimpleMatch: ", match)
            blue_team = match.blue_team
            red_team = match.red_team

            # pregame statistics
            win_team = BLUE if blue_team.win is True else RED
            participants_blue = [{}] * 5
            # already have 1 bot-lane?
            flag = 4
            for participant in blue_team.participants:
                participant_info = {}
                participant_champion = participant.champion
                participant_summoner = participant.summoner
                participant_region = participant_summoner.region
                champion_mastery = get_champion_mastery(participant_summoner, participant_champion, participant_region)
                participant_info["Champion"] = participant_champion
                participant_info["ChampionMastery"] = champion_mastery.points
                participant_info["SummonerSpellD"] = participant.summoner_spell_d
                participant_info["SummonerSpellF"] = participant.summoner_spell_f
                participant_info["SummonerName"] = participant.summoner
                participant_info["Stats"] = participant.stats
                if len(participant.timeline.frames) >= 10:
                    participant_info["TimeFrame"] = participant.timeline.frames[10]
                    participant_info["GoldTen"] = participant.timeline.frames[10].gold_earned
                    participant_info["CreepTen"] = participant.timeline.frames[10].creep_score
                    participant_info["AvgDmgTakenZeroToTen"] = participant.timeline.damage_taken_per_min_deltas['0-10']
                    participant_info["AvgCrpZeroToTen"] = participant.timeline.creeps_per_min_deltas['0-10']
                    participant_info["AvgGoldZeroToTen"] = participant.timeline.gold_per_min_deltas['0-10']
                    participant_info["AvgXpZeroToTen"] = participant.timeline.xp_per_min_deltas['0-10']
                else:
                    self.valid = False
                    return
                if participant.summoner.id == player.id:
                    self.isWin = True if win_team == BLUE else False
                    self.myTeam = BLUE
                if participant.lane == Lane.top_lane or participant.lane == Lane.mid_lane or participant.lane == Lane.jungle:
                    participants_blue[lane_index[participant.lane]] = participant_info
                else:
                    if participant.role == Role.adc:
                        participants_blue[3] = participant_info
                    elif participant.role == Role.support:
                        participants_blue[4] = participant_info
                    else:
                        participants_blue[flag] = participant_info
                        flag = 3

            # participants_blue.append(participant_info)
            flag = 4
            participants_red = [{}] * 5
            for participant in red_team.participants:
                participant_info = {}
                participant_champion = participant.champion
                participant_summoner = participant.summoner
                participant_region = participant_summoner.region
                champion_mastery = get_champion_mastery(participant_summoner, participant_champion, participant_region)
                participant_info["Champion"] = participant_champion
                participant_info["ChampionMastery"] = champion_mastery.points
                participant_info["SummonerSpellD"] = participant.summoner_spell_d
                participant_info["SummonerSpellF"] = participant.summoner_spell_f
                participant_info["SummonerName"] = participant.summoner
                participant_info["Stats"] = participant.stats
                if len(participant.timeline.frames) >= 10:
                    participant_info["TimeFrame"] = participant.timeline.frames[10]
                    participant_info["GoldTen"] = participant.timeline.frames[10].gold_earned
                    participant_info["CreepTen"] = participant.timeline.frames[10].creep_score
                    participant_info["AvgDmgTakenZeroToTen"] = participant.timeline.damage_taken_per_min_deltas['0-10']
                    participant_info["AvgCrpZeroToTen"] = participant.timeline.creeps_per_min_deltas['0-10']
                    participant_info["AvgGoldZeroToTen"] = participant.timeline.gold_per_min_deltas['0-10']
                    participant_info["AvgXpZeroToTen"] = participant.timeline.xp_per_min_deltas['0-10']
                else:
                    self.valid = False
                    return
                if participant.summoner.id == player.id:
                    self.isWin = True if win_team == RED else False
                    self.myTeam = RED
                if participant.lane == Lane.top_lane or participant.lane == Lane.mid_lane or participant.lane == Lane.jungle:
                    participants_red[lane_index[participant.lane]] = participant_info
                else:
                    if participant.role == Role.adc:
                        participants_red[3] = participant_info
                    elif participant.role == Role.support:
                        participants_red[4] = participant_info
                    else:
                        participants_red[flag] = participant_info
                        flag = 3
            for participant in participants_blue:
                if len(participant) == 0:
                    self.valid = False
                    return
            for participant in participants_red:
                if len(participant) == 0:
                    self.valid = False
                    return
            # decide which team the player belong to
            if self.myTeam == BLUE:
                self.myTeamStat = blue_team
                self.enemyTeamStat = red_team
                self.myTeamParticipants = participants_blue
                self.enemyTeamParticipants = participants_red
                self.first_blood = 1 if blue_team.first_blood else 0
                self.first_inhibitor = 1 if blue_team.first_inhibitor else 0
                self.dragon_kill_my = blue_team.dragon_kills
                self.dragon_kill_enemy = red_team.dragon_kills
            else:
                self.myTeamStat = red_team
                self.enemyTeamStat = blue_team
                self.myTeamParticipants = participants_red
                self.enemyTeamParticipants = participants_blue
                self.first_blood = 1 if red_team.first_blood else 0
                self.first_inhibitor = 1 if red_team.first_inhibitor else 0
                self.dragon_kill_my = red_team.dragon_kills
                self.dragon_kill_enemy = blue_team.dragon_kills

            # data of first dragon
            if self.myTeamStat.first_dragon:
                self.first_dragon = 1
            elif self.enemyTeamStat.first_dragon:
                self.first_dragon = 0
            else:
                self.first_dragon = -1

            # data of first baron
            if self.myTeamStat.first_baron:
                self.first_baron = 1
            elif self.enemyTeamStat.first_baron:
                self.first_baron = 0
            else:
                self.first_baron = -1

            # data of first tower
            if self.myTeamStat.first_tower:
                self.first_tower = 1
            elif self.enemyTeamStat.first_tower:
                self.first_tower = 0
            else:
                self.first_tower = -1

        except Exception as e:
            self.valid = False
            return
    # convert to vector(array)
    def pregame_to_vector(self):
        if not self.valid:
            return None
        res = []
        res.append(1 if self.isWin else 0)
        for participant in self.myTeamParticipants:
            res.append(self.all_champions.index(participant["Champion"]) + 1)
        for participant in self.enemyTeamParticipants:
            res.append(self.all_champions.index(participant["Champion"]) + 1)
        for participant in self.myTeamParticipants:
            res.append(participant["ChampionMastery"])
        for participant in self.enemyTeamParticipants:
            res.append(participant["ChampionMastery"])
        for participant in self.myTeamParticipants:
            res.append(self.all_spells.index(participant["SummonerSpellD"]) + 1)
        for participant in self.enemyTeamParticipants:
            res.append(self.all_spells.index(participant["SummonerSpellD"]) + 1)
        for participant in self.myTeamParticipants:
            res.append(self.all_spells.index(participant["SummonerSpellF"]) + 1)
        for participant in self.enemyTeamParticipants:
            res.append(self.all_spells.index(participant["SummonerSpellF"]) + 1)
        return res

    def ingame_to_vector(self):
        # format:
        # |damage|kda|gold|...|average gold|standard gold deviation|first blood|first inhibitor|first tower|first dragon|first baron|dragon kill| <- (blue)
        # |damage|kda|gold|...|average gold|standard gold deviation|first blood|first inhibitor|first tower|first dragon|first baron|dragon kill| <- (red)
        # |first blood|first inhibitor| <- (game) [1 for blue, 0 for red]
        if not self.valid:
            return None
        stat_vec = []
        stat_vec.append(1 if self.isWin else 0)
        gold_blue = 0
        gold_red = 0
        gold_blue_standard = 0
        gold_red_standard = 0
        # calculate my team stats
        # damage kda gold
        for participant in self.myTeamParticipants:
            stat_vec.append(participant["Stats"].total_damage_dealt)
            stat_vec.append(participant["Stats"].kda)
            stat_vec.append(participant["Stats"].gold_earned)
            gold_blue += participant["Stats"].gold_earned
        # avg gold
        gold_blue_mean = gold_blue / len(self.myTeamParticipants)
        stat_vec.append(gold_blue_mean)
        for participant in self.myTeamParticipants:
            gold_blue_standard += math.pow(participant["Stats"].gold_earned - gold_blue_mean, 2)
        gold_blue_standard = math.sqrt(gold_blue_standard)
        # standard gold deviation
        stat_vec.append(gold_blue_standard)
        # data of if the team gets the first things
        stat_vec.append(1 if self.first_blood == 1 else 0)
        stat_vec.append(1 if self.first_inhibitor == 1 else 0)
        stat_vec.append(1 if self.first_tower == 1 else 0)
        stat_vec.append(1 if self.first_dragon == 1 else 0)
        stat_vec.append(1 if self.first_baron == 1 else 0)
        stat_vec.append(self.dragon_kill_my)

        # calculate enemy team stats
        for participant in self.enemyTeamParticipants:
            stat_vec.append(participant["Stats"].total_damage_dealt)
            stat_vec.append(participant["Stats"].kda)
            stat_vec.append(participant["Stats"].gold_earned)
            gold_red += participant["Stats"].gold_earned
        gold_red_mean = gold_red / len(self.enemyTeamParticipants)
        stat_vec.append(gold_red_mean)
        for participant in self.enemyTeamParticipants:
            gold_red_standard += math.pow(participant["Stats"].gold_earned - gold_red_mean, 2)
        gold_red_standard = math.sqrt(gold_red_standard)
        stat_vec.append(gold_red_standard)
        # fdata of if the team gets the first things
        stat_vec.append(1 if self.first_blood == 0 else 0)
        stat_vec.append(1 if self.first_inhibitor == 0 else 0)
        stat_vec.append(1 if self.first_tower == 0 else 0)
        stat_vec.append(1 if self.first_dragon == 0 else 0)
        stat_vec.append(1 if self.first_baron == 0 else 0)
        stat_vec.append(self.dragon_kill_enemy)

        # game stats
        # stat_vec.append(self.first_blood)
        # stat_vec.append(self.first_inhibitor)

        return stat_vec

    def earlygame_to_vec(self):
        if not self.valid:
            return None
        res = []
        res.append(1 if self.isWin else 0)
        for i in range(0, self.player_number_perTeam):
            res.append(self.myTeamParticipants[i]["GoldTen"] - self.enemyTeamParticipants[i]["GoldTen"])
            res.append(self.myTeamParticipants[i]["CreepTen"] - self.enemyTeamParticipants[i]["CreepTen"])
            res.append(self.myTeamParticipants[i]["AvgDmgTakenZeroToTen"] - self.enemyTeamParticipants[i]["AvgDmgTakenZeroToTen"])
            res.append(self.myTeamParticipants[i]["AvgCrpZeroToTen"] - self.enemyTeamParticipants[i]["AvgCrpZeroToTen"])
            res.append(self.myTeamParticipants[i]["AvgGoldZeroToTen"] - self.enemyTeamParticipants[i]["AvgGoldZeroToTen"])
            res.append(self.myTeamParticipants[i]["AvgXpZeroToTen"] - self.enemyTeamParticipants[i]["AvgXpZeroToTen"])

        return res

    # helper function
    def record_in_vec(self, vec, start_pos, participant):
        vec[start_pos + self.all_champions.index(participant["Champion"])] = 1
        start_pos += self.total_champions_number
        vec[start_pos] = participant["ChampionMastery"]
        start_pos += 1
        vec[start_pos + (
            self.all_spells.index(participant["SummonerSpellD"])) * self.total_spells_number + self.all_spells.index(
            participant["SummonerSpellF"]) - 1] = 1
        start_pos += self.total_spells_number * (self.total_spells_number)
        return start_pos

    def parse_vec(self, vec):
        for i in range(0, len(vec)):
            if vec[i] > 0:
                print(str(i) + " : " + str(vec[i]))

SimpleMatch may seems really long and not simple at all, but actually it just record those fields in a easy-to-use way.

The function that really connect the SimpleMatch with our model training is pregame_to_vec()/ ingame_to_vec()/ earlygame_to_vec() which make our match data even more compressive, we convert our match data in the following structure (It's important. The data format is used in the prediction section) :

### pregame
|Champion ID (player 1)|...|Champion ID (player 10)|  
|Champion Mastery (player 1)|...|Champion Mastery (player 10)|  
|Summoner Spell D (player 1)|...|Summoner Spell D (player 10)|   
|Summoner Spell F (player 1)|...|Summoner Spell F (player 10)|   

### ingame
|damage|kda|gold|<-(player 1)... |damage|kda|gold|<-(player 5)|average gold|standard gold deviation|first blood|first inhibitor|first tower|first dragon|first baron|dragon kill| <- (blue team)  
|damage|kda|gold|<-(player 6)... |damage|kda|gold|<-(player 10)|average gold|standard gold deviation|first blood|first inhibitor|first tower|first dragon|first baron|dragon kill| <- (red team)  

### earlygame
|player 1's gold at 10 minutes - enemy 1's gold at 10 minutes|  
|player 1's creep at 10 minutes - enemy 1's creep at 10 minutes|  
|player 1's average damage taken in 10 minutes - enemy 1's average damage taken in 10 minutes|  
|player 1's average creep in 10 minutes - enemy 1's average creep in 10 minutes|  
|player 1's average gold in 10 minutes - enemy 1's average gold in 10 minutes|  
|player 1's average xp in 10 minutes - enemy 1's average xp in 10 minutes|  
...  
|player 5's gold at 10 minutes - enemy 5's gold at 10 minutes|  
|player 5's total creep in 10 minutes - enemy 5's total creep in 10 minutes|  
|player 5's average damage taken in 10 minutes - enemy 5's average damage taken in 10 minutes|  
|player 5's average creep in 10 minutes - enemy 5's average creep in 10 minutes|  
|player 5's average gold in 10 minutes - enemy 5's average gold in 10 minutes|  
|player 5's average xp in 10 minutes - enemy 5's average xp in 10 minutes|  

### ps
1. Enemy1 is the same as player6 since there are 2 teams in each match and each team have 5 player.  

2. In data processing, we have matched each player's role, for instance, player 1 will compare to player 6(aka enemy 1), so their role must be the same. It's meaningless if the player 1 is a fighter and player 6 is a supporter. So when we compare two player's data, we already make sure they are the same role (in the init function)  

#### Some helper functions:

In [5]:
def fetch_match_history(start, end):
    kalturi = Summoner(name="Kalturi", region="NA")
    his = kalturi.match_history
    # length check removed because len(his) will take multiple calls to complete
    # which takes to much time
    # if match_num >= len(his):
    #     match_num = len(his) - 1
    # construct 2D list
    pregame_vec = []
    ingame_vec = []
    earlygame_vec = []
    for i in range(start, end):
        testMatch = SimpleMatch(his[i], kalturi)
        vec = testMatch.pregame_to_vector()
        if vec is not None:
            pregame_vec.append(vec)
        vec = testMatch.ingame_to_vector()
        if vec is not None:
            ingame_vec.append(vec)
        vec = testMatch.earlygame_to_vec()
        if vec is not None:
            earlygame_vec.append(vec)
        # print(testMatch.ingame_to_vector())
        print("match " + str(i) + " recorded")
    # save list to csv file
    write = pd.DataFrame(data=pregame_vec)
    write.to_csv('pregame10.csv', encoding='utf-8')
    write = pd.DataFrame(data=ingame_vec)
    write.to_csv('ingame10.csv', encoding='utf-8')
    write = pd.DataFrame(data=earlygame_vec)
    write.to_csv('earlygame10.csv', encoding='utf-8')

def read_csv_to_ndarray(filename):
    # read csv file to ndarray
    read = pd.read_csv(filename)
    target_ndarray = read.values
    target_ndarray_x = np.delete(target_ndarray, 0, 1)
    target_ndarray_y = target_ndarray_x[:, 0]
    target_ndarray_x = np.delete(target_ndarray_x, 0, 1)
    return target_ndarray_x, target_ndarray_y.T

def combine_two_csv(filename1, filename2):
    read1 = pd.read_csv(filename1)
    read2 = pd.read_csv(filename2)
    target_ndarray1 = np.delete(read1.values, 0, 1)
    target_ndarray2 = np.delete(read2.values, 0, 1)
    res = np.concatenate((target_ndarray1, target_ndarray2), axis=0)
    write = pd.DataFrame(data=res)
    write.to_csv(filename1.replace('.csv', '') + '_' + filename2.replace('.csv', '') + '.csv', encoding='utf-8')


**The code provided cannot run directly since you may not have permission to the Riot API.**  
But if you interested in the code, feel free to get a Riot API key, it's free.  
The main function is provided below.  

In [4]:
if __name__ == "__main__":
    #fetch_match_history(0, 100)
    #print("end")
    pass

Also the final result data (csv format) are attached in the project folder, feel free to check them.
We have saved the result data into "pregame0_pregame1.csv", "earlygame0_earlygame1.csv" and "ingame0_ingame1.csv".